# Extract Covid data using API and save it as a csv file after cleanning.

In [2]:
import requests
import json
import requests
import pandas as pd

API_KEY = '+0GTkxg1mbgtx2GOnjVSxg==FMVkYlqa0X0ocvPm'

In [3]:
import requests
import pandas as pd
import json

def fetch_covid_data(country):
    api_url = 'https://api.api-ninjas.com/v1/covid19?country={}'.format(country)
    response = requests.get(api_url, headers={'X-Api-Key': API_KEY})
    
    if response.status_code == requests.codes.ok:
        print("Succeeded")
        json_object = response.json()
        
        all_cases = []
        for obj in json_object:
            cases = obj['cases']
            all_cases.append(cases)

        combined_cases = {}
        for cases in all_cases:
            for date, values in cases.items():
                if date in combined_cases:
                    combined_cases[date]['total'] += values['total']
                    combined_cases[date]['new'] += values['new']
                else:
                    combined_cases[date] = {'total': values['total'], 'new': values['new']}

        df_combined = pd.DataFrame.from_dict(combined_cases, orient='index')
        return df_combined
    else:
        print("Error:", response.status_code, response.text)
        return None

In [4]:
# Example usage:
country_name = 'United States'  # Replace with any country you want to fetch data for
US = fetch_covid_data(country_name)
if US is not None:
    print(US.head())

Succeeded
            total  new
2020-01-22      1    0
2020-01-23      1    0
2020-01-24      2    1
2020-01-25      2    0
2020-01-26      5    3


In [5]:
# Example usage:
country_name = 'Canada'  # Replace with any country you want to fetch data for
CA = fetch_covid_data(country_name)
if CA is not None:
    print(CA.head())

Succeeded
            total  new
2020-01-22      0    0
2020-01-23      2    2
2020-01-24      3    1
2020-01-25      3    0
2020-01-26      3    0


In [21]:
# Fetch COVID-19 data for Canada
CA = fetch_covid_data('Canada')

# Fetch COVID-19 data for the United States
US = fetch_covid_data('United States')

# Concatenate the data for Canada and the United States
na_covid_data = pd.concat([CA, US], keys=['Canada', 'United States'], names=['Country'])

# Merge the data for each date and aggregate the total and new cases
na_covid_data_merged = na_covid_data.groupby(level=1).sum()

# Reset the index and rename the index column to 'Date'
na_covid_data_merged.reset_index(inplace=True)
na_covid_data_merged.rename(columns={'index': 'Date'}, inplace=True)

# Print the merged DataFrame
print(na_covid_data_merged.head())

# Export the merged DataFrame to a CSV file
na_covid_data_merged.to_csv('na_covid_data_merged.csv')


Succeeded
Succeeded
         Date  total  new
0  2020-01-22      1    0
1  2020-01-23      3    2
2  2020-01-24      5    2
3  2020-01-25      5    0
4  2020-01-26      8    3


In [20]:
na_covid_data_merged

,total,new
2020-01-22,1,0
2020-01-23,3,2
2020-01-24,5,2
2020-01-25,5,0
2020-01-26,8,3
...,...,...
2023-03-05,108256726,-3569
2023-03-06,108266967,10241
2023-03-07,108303129,36162
2023-03-08,108368825,65696


In [24]:
covid_data = pd.read_csv('./na_covid_data_merged.csv', index_col=0)

In [25]:
covid_data

,Date,total,new
0,2020-01-22,1,0
1,2020-01-23,3,2
2,2020-01-24,5,2
3,2020-01-25,5,0
4,2020-01-26,8,3
...,...,...,...
1138,2023-03-05,108256726,-3569
1139,2023-03-06,108266967,10241
1140,2023-03-07,108303129,36162
1141,2023-03-08,108368825,65696
